# Import requirements

In [1]:
import os
import pdb
import argparse
import re
import torch
from torch.nn.utils.rnn import pad_sequence
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 1. Preprocess

In [4]:
#데이터 클리닝
train_data = pd.read_csv('train.csv', index_col='index')
test_data = pd.read_csv('test.csv', index_col='index')
train_data.head()

,category,data
index,,
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [5]:
test_data.head()

,data
index,
0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...
1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...
2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요
3,국민위원에가 삼성편만들어요. 삼성에서 11년간 일하고 혈암과 백혈병 진단을 받은 ...
4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부..."


In [6]:
train_data.shape, test_data.shape

((40000, 2), (5000, 1))

In [7]:
def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [8]:
train_data.data = clean_text(train_data.data)
test_data.data = clean_text(test_data.data)

In [9]:
train_data_text = list(train_data['data'])

train_clear_text = []

for i in tqdm(range(len(train_data_text))):
  train_clear_text.append(str(train_data_text[i]).replace('\\n', ''))
train_data['clear_text'] = train_clear_text
train_data.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:00<00:00, 583788.85it/s]


,category,data,clear_text
index,,,
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올...","빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,1,"티비 세,세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠시 딴일...","티비 세,세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠시 딴일..."
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [10]:
train_clear_text = list(train_data['clear_text'])

train_clear_text2 = []

for text in train_clear_text:
  temp = re.sub('[-=+,#:;//●<>▲\?:^$.☆!★()Ⅰ@*\"※~>`\'…》]', ' ', text)
  train_clear_text2.append(temp)
train_data['clear_text'] = train_clear_text2
train_data.head()

,category,data,clear_text
index,,,
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...,신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...,학교이름에 남자 도 붙여주세요 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올...",빙상연맹 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요 최근 동계올...
3,1,"티비 세,세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠시 딴일...",티비 세 세 관람가도 연령확인 의무화 하자 제기 에전에 티비를 보다가 잠시 딴일...
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...,무더운 여름철엔 남성들도 시원한 자율복장을 해야 무더운 여름철에는 남성들도 노넥타...


In [11]:
test_data_text = list(test_data['data'])

test_clear_text = []

for i in tqdm(range(len(test_data_text))):
  test_clear_text.append(test_data_text[i].replace('\\n', ' '))
test_data['clear_text'] = test_clear_text
test_data.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 370331.81it/s]


,data,clear_text
index,,
0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이 법으로 인해 보호받지 못하...
1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에 *지역 어린이...
2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요
3,국민위원에가 삼성편만들어요. 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...,국민위원에가 삼성편만들어요. 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...
4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부...","방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부..."


In [12]:
test_clear_text = list(test_data['clear_text'])

test_clear_text2 = []

for text in test_clear_text:
  temp = re.sub('[-=+,#:;//●<>▲\?:^$.☆!★()Ⅰ@*\"※~>`\'…》]', ' ', text)
  test_clear_text2.append(temp)
test_data['clear_text'] = test_clear_text2
test_data.head()

,data,clear_text
index,,
0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...,소년법 폐지해주세요 법 아래에서 보호받아야 할 아이들이 법으로 인해 보호받지 못하...
1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...,국공립 유치원 증설에 관하여 국공립 유치원 부지 학보와건립및 증설에 지역 어린이...
2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요,나경원파면 나경원의원의 동계올림픽 위원을 파면해 주세요
3,국민위원에가 삼성편만들어요. 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...,국민위원에가 삼성편만들어요 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...
4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부...",방과후 유치원 어린이집 영어교육을 유지시켜주세요 저는 아이 셋 키우는 평범한 주부...


In [13]:
train_data =  train_data[[ 'clear_text', 'category']]
# train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=0)

test_data = test_data[[ 'clear_text']]

In [14]:
train_data = train_data.values.tolist()
# valid_data = valid_data.values.tolist()

In [15]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/jupyter/Untitled Folder/.cache/kobert_v1.zip
using cached model. /home/jupyter/Untitled Folder/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
class SubmitDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.dataset = dataset
        self.sentences = [transform([i[sent_idx]]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.dataset))

In [18]:
# Setting parameters
max_len = 300
batch_size = 8
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  1e-5

In [19]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
# data_valid = BERTDataset(valid_data, 0, 1, tok, max_len, True, False)

using cached model. /home/jupyter/Untitled Folder/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=4)

In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # model.eval()
    # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    #     token_ids = token_ids.long().to(device)
    #     segment_ids = segment_ids.long().to(device)
    #     valid_length= valid_length
    #     label = label.long().to(device)
    #     out = model(token_ids, valid_length, segment_ids)
    #     test_acc += calc_accuracy(out, label)
    # print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/5000 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7780145406723022 train acc 0.375
epoch 1 batch id 201 loss 1.8992738723754883 train acc 0.23569651741293532
epoch 1 batch id 401 loss 1.2781410217285156 train acc 0.31889027431421446
epoch 1 batch id 601 loss 0.7527196407318115 train acc 0.44363560732113144
epoch 1 batch id 801 loss 0.35072165727615356 train acc 0.5383895131086143
epoch 1 batch id 1001 loss 0.6468955278396606 train acc 0.599025974025974
epoch 1 batch id 1201 loss 0.2658335566520691 train acc 0.6413405495420483
epoch 1 batch id 1401 loss 0.29316332936286926 train acc 0.6728229835831548
epoch 1 batch id 1601 loss 0.31189969182014465 train acc 0.6960493441599
epoch 1 batch id 1801 loss 0.057994481176137924 train acc 0.7157828983897835
epoch 1 batch id 2001 loss 0.5219630599021912 train acc 0.7312593703148426
epoch 1 batch id 2201 loss 0.2118060290813446 train acc 0.743071331213085
epoch 1 batch id 2401 loss 0.04615937918424606 train acc 0.7546855476884632
epoch 1 batch id 2601 loss 0.080563344061

  0%|          | 0/5000 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.7464577555656433 train acc 0.75
epoch 2 batch id 201 loss 0.018634889274835587 train acc 0.8812189054726368
epoch 2 batch id 401 loss 0.006774192675948143 train acc 0.885286783042394
epoch 2 batch id 601 loss 0.4154807925224304 train acc 0.879783693843594
epoch 2 batch id 801 loss 0.0675763338804245 train acc 0.8818664169787765
epoch 2 batch id 1001 loss 0.6116683483123779 train acc 0.8838661338661339
epoch 2 batch id 1201 loss 0.04525710642337799 train acc 0.8863447127393839
epoch 2 batch id 1401 loss 0.17426781356334686 train acc 0.8869557458957887
epoch 2 batch id 1601 loss 0.08780013024806976 train acc 0.889053716427233
epoch 2 batch id 1801 loss 0.016048289835453033 train acc 0.8916574125485841
epoch 2 batch id 2001 loss 0.5465800762176514 train acc 0.8935532233883059
epoch 2 batch id 2201 loss 0.009228540584445 train acc 0.8944229895502045
epoch 2 batch id 2401 loss 0.011649742722511292 train acc 0.8959287796751354
epoch 2 batch id 2601 loss 0.0645509809

  0%|          | 0/5000 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.6360301375389099 train acc 0.875
epoch 3 batch id 201 loss 0.009105350822210312 train acc 0.9172885572139303
epoch 3 batch id 401 loss 0.0035597248934209347 train acc 0.9189526184538653
epoch 3 batch id 601 loss 0.2783946692943573 train acc 0.9130615640599001
epoch 3 batch id 801 loss 0.1256592571735382 train acc 0.9151061173533084
epoch 3 batch id 1001 loss 0.4825160503387451 train acc 0.9150849150849151
epoch 3 batch id 1201 loss 0.028411902487277985 train acc 0.9167360532889259
epoch 3 batch id 1401 loss 0.024138817563652992 train acc 0.9178265524625268
epoch 3 batch id 1601 loss 0.1714656800031662 train acc 0.9184884447220487
epoch 3 batch id 1801 loss 0.008103941567242146 train acc 0.9200444197667962
epoch 3 batch id 2001 loss 0.4906173348426819 train acc 0.9211019490254873
epoch 3 batch id 2201 loss 0.008113402873277664 train acc 0.9214561562925943
epoch 3 batch id 2401 loss 0.004745178855955601 train acc 0.9225322782174095
epoch 3 batch id 2601 loss 0.0

In [24]:
model.eval()
test_dataset = test_data['clear_text'].values.tolist()
dataset_test = SubmitDataset(test_dataset, 0, tok, max_len, True, False)
test_dataloader = DataLoader(dataset_test, batch_size = 2, num_workers=2)
result = []
for batch_id, (token_ids, valid_length, segment_ids) in enumerate(test_dataloader):
    
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)
    

    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()

        pred = np.argmax(logits).tolist()
        result.append(pred)


submit = {'index' : list(range(len(result))), 'category' : result}

df = pd.DataFrame(submit)
df.to_csv('submit.csv', index=False)

            